In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load Mistral Model (already downloaded)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("✅ Mistral model loaded.")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ Mistral model loaded.


In [2]:
import pandas as pd
from tqdm import tqdm

# Load CSV with 'line' and 'stereotype_type' columns from Phase 2
df = pd.read_csv("chunk_0_250.csv")

# Reuse your function for rewriting
def rewrite_biased_line_with_mistral_v2(line, stereotype_type):
    prompt = f"""
You are a helpful assistant trained to rewrite movie script lines that reinforce gender stereotypes.

Stereotype detected: {stereotype_type}

Avoid rephrasing the same stereotype. Instead, remove the biased framing and rewrite the line by giving the character agency, ambition, or equality, while preserving narrative meaning.

Line: "{line}"
Rewritten line:"""

    inputs = tokenizer(prompt.strip(), return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=40, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Extract only the rewritten line
    rewritten_line = decoded.split("Rewritten line:")[-1].strip().strip('"')
    return rewritten_line

# Enable progress bar
tqdm.pandas()

# Apply rewrites only where stereotype_type is NOT "none"
df["rewritten_line"] = df.progress_apply(
    lambda row: rewrite_biased_line_with_mistral_v2(row["line"], row["stereotype_type"])
    if row["stereotype_type"] != "none" else row["line"],
    axis=1
)

# Save the Phase 4A output
df.to_csv("phase4a_rewrites.csv", index=False)
print("✅ Saved Phase 4A output to phase4a_rewrites.csv")


100%|██████████| 250/250 [27:49<00:00,  6.68s/it]

✅ Saved Phase 4A output to phase4a_rewrites.csv
